In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/usr/local/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Perceptron

In [9]:
x = tf.constant([[0.7, 0.9]])

# w1 = tf.get_variable('w1', shape=[2, 3], initializer=tf.random_normal_initializer, dtype=tf.float32)
# w2 = tf.get_variable('w2', shape=[3, 1], initializer=tf.random_normal_initializer, dtype=tf.float32)
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

In [8]:
tf.trainable_variables()

[<tf.Variable 'w1:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'w2:0' shape=(3, 1) dtype=float32_ref>]

In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
sess.run(y)
sess.close()

array([[3.957578]], dtype=float32)

## 乘法

In [11]:
v1 = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
v2 = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)

In [15]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

sess.run(tf.matmul(v1, v2))
sess.run(v1 * v2)
sess.run(tf.multiply(v1, v2))
sess.close()

array([[19., 22.],
       [43., 50.]], dtype=float32)

array([[ 5., 12.],
       [21., 32.]], dtype=float32)

array([[ 5., 12.],
       [21., 32.]], dtype=float32)

## 自定损失函数

In [43]:
dataset_size = 10000
rdm = np.random.RandomState(2018)
x_input = rdm.rand(dataset_size, 2)
y_input = [[x1 + x2 + rdm.rand()/10-0.05] for [x1, x2] in x_input]

In [51]:
bs = 128
x = tf.placeholder(tf.float32, [None, 2], 'x-input')
y_ = tf.placeholder(tf.float32, [None, 1], 'y-input')

w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=2018))
y = tf.matmul(x, w1)

loss_more = 1
loss_less = 1
loss = tf.reduce_sum(tf.where(tf.greater(y, y_), loss_more*(y-y_), loss_less*(y_-y)))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [52]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())

bm = dataset_size // bs
for epoch in range(100):
    for i in range(bm):
        start = i * bs
        end = min((i+1) * bs, dataset_size)
        sess.run(train_step, feed_dict={x:x_input[start:end], y_:y_input[start:end]})
    if epoch % 9 == 0:
        sess.run(w1)
sess.close()

array([[ 1.9920378 ],
       [-0.82629216]], dtype=float32)

array([[ 1.9998435 ],
       [-0.21224669]], dtype=float32)

array([[1.4918191 ],
       [0.40767875]], dtype=float32)

array([[1.0155504],
       [0.9780425]], dtype=float32)

array([[0.99900526],
       [0.9967638 ]], dtype=float32)

...

array([[0.9990892],
       [0.9968427]], dtype=float32)

array([[0.99923944],
       [0.99719816]], dtype=float32)

array([[0.9994898],
       [0.9973638]], dtype=float32)

array([[0.99960035],
       [0.9975564 ]], dtype=float32)

array([[0.999527 ],
       [0.9976091]], dtype=float32)

## 可变学习率

In [ ]:
global_step = tf.Variable(0)

lr = tf.train.exponential_decay(0.1, global_step=global_step, decay_steps=100, decay_rate=0.96, staircase=True) # 每100轮lr乘0.96

train_step = tf.train.GradientDescentOptimizer(lr).minimize(loss, global_step=global_step)

## 正则化

tf.nn ：提供神经网络相关操作的支持，包括卷积操作（conv）、池化操作（pooling）、归一化、loss、分类操作、embedding、RNN、Evaluation。
tf.layers：主要提供的高层的神经网络，主要和卷积相关的，是对tf.nn的进一步封装，tf.nn会更底层一些。
tf.contrib：tf.contrib.layers提供够将计算图中的  网络层、正则化、摘要操作、是构建计算图的高级操作，但是tf.contrib包含不稳定和实验代码，有可能以后API会改变。

以上三个模块的封装程度是逐个递进的。 

In [ ]:
weight = tf.Variable()
y = tf.matmul(x, weight)

tf.contrib.layers.l1_regularizer()(weight)
tf.contrib.layers.l2_regularizer()(weight)
tf.contrib.layers.l1_l2_regularizer()(weight)

## 集合

In [ ]:
tf.add_to_collection()
tf.get_collection()

## 滑动平均模型

In [53]:
v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)

ema = tf.train.ExponentialMovingAverage(0.99, num_updates=step)
main_average_op = ema.apply([v1])

In [59]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())

sess.run([v1, ema.average(v1)])

_ = sess.run(tf.assign(v1, 5))
sess.run(main_average_op)
sess.run([v1, ema.average(v1)])

_ = sess.run(tf.assign(step, 10000))
_ = sess.run(tf.assign(v1, 10))
sess.run(main_average_op)
sess.run([v1, ema.average(v1)])

sess.run(main_average_op)
sess.run([v1, ema.average(v1)])
sess.close()

[0.0, 0.0]

[5.0, 4.5]

[10.0, 4.555]

[10.0, 4.60945]

In [ ]:
tf.train.ExponentialMovingAverage